# Spam Email Detection
## Fannie Yuen

### Abstract
Spams can be detected through natural language processing and machine learning methodologies. 
Research on spam email detection either focuses on natural language processing methodologies on single machine learning algorithm or one natural language processing technique on multiple machine learning algorithms.
In this notebook, a modeling pipeline is developed to review both the natural language processing techniques and the machine learning methodologies. 
Modeling results show that preprocessing the emails by most-frequent-word-count modeling in logistic regression gives the best model performance.

### Introduction

Email has been used as a modern communication method due to its simplicity and cost-effectiveness. The number of email users worldwide is forecasted to rise to 2.9 billion by 2019. Nearly 105 billion emails are sent each day. This number is expected to reach 246 billion before 2020$^1$. However, spam emails received by recipients are not desirable. Traditional methods of spam filtering such as blacklists and whitelists using mailing addresses, domains, IP addresses are not effective. Alternatively, spam email detection based on machine learning is an application to reduce spam email. 

In this notebook, a spam email detection classifier will be developed. The following key topics will be covered. First, the data$^2$ will be handled through `email` and `BeautifulSoup` libraries. Text preprocessing with and without **Natural Language Processing (NLP) concerning email content** will be evaluated. A number of **machine learning methodologies** are introduced to detect spam email from the SpamAssassin$^3$, an open source anti-spam platform. Moreover, identification of model **underfitting and overfitting** will be discussed and model performances will be reviewed through cross-validation and evaluation metrics. 

The development is implemented in **Python 3**. Some codes follow the existing kernel$^4$.

The workflow of developing a spam email detection classifier is as follows:

1. Data Reading and Inspection
2. Text Preprocessing
3. Modeling
4. Results
5. Conclusions
6. Improvements
7. Appendix

Now, let us start with loading the libraries.

In [1]:
# load libraries
import pandas as pd
import numpy as np
import os
import warnings
import email
import email.policy
from bs4 import BeautifulSoup
import re

from collections import Counter

import nltk

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix

from sklearn.pipeline import Pipeline

from sklearn.dummy import DummyClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_score, train_test_split

### 1. Data Reading and Inspection
The `email` package manages email message, which loads the email data. The email data consists of either spams or hams. Spams, aka junk emails, are unsolicited messages sent in bulk by email$^5$. Hams are non-spams expected by email recipients. 

Data is read and inspected according to the existing kernel$^4$. Each file in the data source represents an email message.

In [2]:
ham_filenames = [name for name in sorted(os.listdir('../ham-and-spam-dataset/hamnspam/ham'))]
spam_filenames = [name for name in sorted(os.listdir('../ham-and-spam-dataset/hamnspam/spam'))]

In [3]:
print('Amount of ham files:', len(ham_filenames))
print('Amount of spam files:', len(spam_filenames))    
print('Spam to Ham Ratio: {:.2f}%'.format(100 * len(spam_filenames) / len(ham_filenames)))
print('Spam to All Ratio: {:.2f}%'.format(100 * len(spam_filenames) / (len(ham_filenames) + len(spam_filenames))))
print('Ham to All Ratio: {:.2f}%'.format(100 * len(ham_filenames) / (len(ham_filenames) + len(spam_filenames))))

Amount of ham files: 2551
Amount of spam files: 501
Spam to Ham Ratio: 19.64%
Spam to All Ratio: 16.42%
Ham to All Ratio: 83.58%


In [4]:
%%time
def load_email(is_spam, filename):
    directory = "../ham-and-spam-dataset/hamnspam/spam" if is_spam else "../ham-and-spam-dataset/hamnspam/ham"
    with open(os.path.join(directory, filename), "rb") as f:
        # create BytesParser instance; parse(): parse resulting bytes and return message object
        return email.parser.BytesParser(policy = email.policy.default).parse(f)
   
print("Loading 2 email groups...")
ham_emails = [load_email(is_spam = False, filename = name) for name in ham_filenames]
spam_emails = [load_email(is_spam = True, filename = name) for name in spam_filenames]
print("Emails loaded!")

Loading 2 email groups...
Emails loaded!
CPU times: user 3.52 s, sys: 395 ms, total: 3.92 s
Wall time: 4.39 s


After loading the emails, a preview of a sample email is shown.

In [5]:
# how emails are stored
print('Header Field Names:', ham_emails[1].keys())
print('\n')
print('Message Field Values:', ham_emails[1].values())
print('\n')
print('Message Content:', spam_emails[1].get_content())

Header Field Names: ['Return-Path', 'Delivered-To', 'Received', 'Received', 'Received', 'X-Egroups-Return', 'Received', 'X-Sender', 'X-Apparently-To', 'Received', 'Received', 'Received', 'Received', 'Received', 'Received', 'Message-Id', 'To', 'X-Mailer', 'X-Egroups-From', 'From', 'X-Yahoo-Profile', 'MIME-Version', 'Mailing-List', 'Delivered-To', 'Precedence', 'List-Unsubscribe', 'Date', 'Subject', 'Reply-To', 'Content-Type', 'Content-Transfer-Encoding']


Message Field Values: ['<Steve_Burt@cursor-system.com>', 'zzzz@localhost.netnoteinc.com', 'from localhost (localhost [127.0.0.1])\tby phobos.labs.netnoteinc.com (Postfix) with ESMTP id BE12E43C34\tfor <zzzz@localhost>; Thu, 22 Aug 2002 07:46:38 -0400 (EDT)', 'from phobos [127.0.0.1]\tby localhost with IMAP (fetchmail-5.9.0)\tfor zzzz@localhost (single-drop); Thu, 22 Aug 2002 12:46:38 +0100 (IST)', 'from n20.grp.scd.yahoo.com (n20.grp.scd.yahoo.com    [66.218.66.76]) by dogma.slashnull.org (8.11.6/8.11.6) with SMTP id    g7MBkTZ05087 f

### 2. Text Preprocessing
In this section, the email structure will be extracted and content of the emails will be converted to plain text for the text analysis. This is executed through the functions: `get_email_structure()`, `structures_counter()`, `html_to_plain()` and `email_to_plain()` from the existing kernel$^4$. Two feature sets will be prepared for the modeling task. The `feature set 1` is based on **stopwords + n-gram + tf-idf** (which will be explained in the `feature set 1` section). The `feature set 2` is **most-frequent-word-count** based, developed from the existing kernel$^4$. 

Since the stopwords (the, is, of, a, to, from ..) eliminated from `feature set 1` are those words which occur most frequently from `feature set 2`, so the underlying ideas of these two feature sets are contradicting. Therefore they are divided into two feature sets and their model performances are reviewed separately.


Both feature sets are generated by the **removal of punctuations**, **lower-casing**, and **word stemming**. 

Most of the machine learning algorithms understand numbers only. Vectorization is a process to convert a bag of words into an array of numbers, which is a representation of words. Consider a very long email with say 987,654 words in the training data. If each word is represented as a feature in a vector, then this vector will have a length of 987,654, however, this will introduce lots of zeros for a short email with say 123 words. In order to perform efficient operations, **Compressed Sparse Row (CSR)** represents three 1-D arrays for non-zero values, the extents of rows and the column indices so that features are encoded as **csr matrix** before modeling takes place.

Let's start with getting the email structure.

In [6]:
def get_email_structure(email):
    if isinstance(email, str):
        return email
    # payload is referred as content
    payload = email.get_payload()
    if isinstance(payload, list):
        # multipart: type of payload, a structured sequence of sub-messages each with own set of headers & payload
        return "multipart({})".format(", ".join([
            get_email_structure(sub_email)
            for sub_email in payload
        ]))
    else:
        return email.get_content_type()

def structures_counter(emails):
    structures = Counter()
    for email in emails:
        structure = get_email_structure(email)
        structures[structure] += 1
    return structures

ham_structure = structures_counter(ham_emails)
spam_structure = structures_counter(spam_emails)

In [7]:
ham_structure.most_common(10)

[('text/plain', 2453),
 ('multipart(text/plain, application/pgp-signature)', 72),
 ('multipart(text/plain, text/html)', 8),
 ('multipart(text/plain, text/plain)', 4),
 ('multipart(text/plain)', 3),
 ('multipart(text/plain, application/octet-stream)', 2),
 ('multipart(text/plain, text/enriched)', 1),
 ('multipart(text/plain, application/ms-tnef, text/plain)', 1),
 ('multipart(multipart(text/plain, text/plain, text/plain), application/pgp-signature)',
  1),
 ('multipart(text/plain, video/mng)', 1)]

In [8]:
spam_structure.most_common(10)

[('text/plain', 222),
 ('text/html', 181),
 ('multipart(text/plain, text/html)', 45),
 ('multipart(text/html)', 19),
 ('multipart(text/plain)', 19),
 ('multipart(multipart(text/html))', 5),
 ('multipart(text/plain, image/jpeg)', 3),
 ('multipart(text/html, application/octet-stream)', 2),
 ('multipart(text/plain, application/octet-stream)', 1),
 ('multipart(text/html, text/plain)', 1)]

In [9]:
for email in spam_emails:
    if get_email_structure(email) == 'text/html':
        testEmail = email
        break

print(testEmail.get_content())

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.0 Transitional//EN">
<HTML><HEAD>
<META content="text/html; charset=windows-1252" http-equiv=Content-Type>
<META content="MSHTML 5.00.2314.1000" name=GENERATOR></HEAD>
<BODY><!-- Inserted by Calypso -->
<TABLE border=0 cellPadding=0 cellSpacing=2 id=_CalyPrintHeader_ rules=none 
style="COLOR: black; DISPLAY: none" width="100%">
  <TBODY>
  <TR>
    <TD colSpan=3>
      <HR color=black noShade SIZE=1>
    </TD></TR></TD></TR>
  <TR>
    <TD colSpan=3>
      <HR color=black noShade SIZE=1>
    </TD></TR></TBODY></TABLE><!-- End Calypso --><!-- Inserted by Calypso --><FONT 
color=#000000 face=VERDANA,ARIAL,HELVETICA size=-2><BR></FONT></TD></TR></TABLE><!-- End Calypso --><FONT color=#ff0000 
face="Copperplate Gothic Bold" size=5 PTSIZE="10">
<CENTER>Save up to 70% on Life Insurance.</CENTER></FONT><FONT color=#ff0000 
face="Copperplate Gothic Bold" size=5 PTSIZE="10">
<CENTER>Why Spend More Than You Have To?
<CENTER><FONT color=#ff0000 face="Copp

In [10]:
def html_to_plain(email):
    try:
        # parse data from html email content
        soup = BeautifulSoup(email.get_content(), 'html.parser')
        # return a clear form of text
        return soup.text.replace('\n\n','')
    except:
        return "empty"

print(html_to_plain(testEmail))


Save up to 70% on Life Insurance.
Why Spend More Than You Have To?Life Quote Savings
Ensuring your 
      family's financial security is very important. Life Quote Savings makes 
      buying life insurance simple and affordable. We Provide FREE Access to The 
      Very Best Companies and The Lowest Rates.Life Quote Savings is FAST, EASY and 
            SAVES you money! Let us help you get started with the best values in 
            the country on new coverage. You can SAVE hundreds or even thousands 
            of dollars by requesting a FREE quote from Lifequote Savings. Our 
            service will take you less than 5 minutes to complete. Shop and 
            compare. SAVE up to 70% on all types of Life insurance! Click Here For Your 
            Free Quote!Protecting your family is the best investment you'll ever 
          make!
If you are in receipt of this email 
      in error and/or wish to be removed from our list, PLEASE CLICK HERE AND TYPE REMOVE. If you 
      resi

In [11]:
def email_to_plain(email):
    struct = get_email_structure(email)
    # walk(): iterate over all the parts and subparts of a message object tree
    for part in email.walk():
        partContentType = part.get_content_type()
        if partContentType not in ['text/plain','text/html']:
            continue
        try:
            partContent = part.get_content()
        except: # in case of encoding issues
            partContent = str(part.get_payload())
        if partContentType == 'text/plain':
            return partContent
        else:
            return html_to_plain(part)

print("[ham email sample]: ")        
print(email_to_plain(ham_emails[42]))
print("[spam email sample]: ")
print(email_to_plain(spam_emails[42]))

[ham email sample]: 
Joseph S. Barrera III wrote:

> Chris Haun wrote:
>
>> A LifeGem is a certified, high quality diamond created from the 
>> carbon of your loved one as a memorial to their unique and wonderful 
>> life.
>
>
> Why wait until you're dead? I'm sure there's enough carbon in
> the fat from your typical liposuction job to make a decent diamond.
>
> - Joe
>
Oh, hell - what about excrement? I'd love to be able to say - No, the 
sun doesn't shine out of my ass, but there's the occasional diamond. ;-).

Owen


http://xent.com/mailman/listinfo/fork


[spam email sample]: 

New Page 1
VIAGRA
WITHOUT
A DOCTORS VISIT!!
CLICK
HERE
*Other
Top Medications also available!!
*We
have Doctors on call around the country to view
your information and quickly approve your order.
*Totally
Discreet System allows you to order today and
enjoy your medication tomorrow in most cases.
*Finally
you can try the wonder drug Viagra that
has swept the World without the embarrassment of
having to visit 

#### `feature set 1`: "stopwords + n-grams + tf-idf" 
The `feature set 1` is created by exploring the text structure to exploit the contextual features. In particular, the following operations **stopwords**, **n-grams** and **tf-idf** will be applied.

##### Stopwords
Stopwords are words that appear frequently in text but have little lexical content. Their presence creates wrong signal for prediction. Let us explore the common stopwords from the Natural Language Toolkit `nltk` package, a popular library for natural language processing.

In [12]:
stop_words = nltk.corpus.stopwords.words('english')
stop_words

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

##### Stemming
"Laptop" and "Laptops" are just two forms of the same dictionary lemma. In linguistic morphology and information retrieval, stemming is the process of reducing inflected or derived words into their word stem, base or root form.

In the `nltk` library a commonly used stemmer is PorterStemmer. It is originated from 80's with the aim to reduce words to its common form. Let us start with this basic stemmer. 

In [13]:
stemmer = nltk.PorterStemmer()

for word in ("Living", "Live", "Lives", "Lived"):
        print(word, "=>", stemmer.stem(word))

Living => live
Live => live
Lives => live
Lived => live


##### Normalisation
To ensure that certain form of expressions are in the same notation in the training data, regular expressions are used to detect word patterns. These include extracting email addresses, Web URL, amount of money starting with curreny symbol followed by numbers, phone numbers, numbers, punctuation marks, and whitespaces and grouping them to the same terms. This also includes returning all characters to lower case.

Now let us include all the above considerations in a function `transformation()`.

In [14]:
def transformation(X):
        X_to_words = []
        for email in X:
            text = email_to_plain(email)
            
            try:
                text = text.replace("\n", " ")
            except:
                pass
            
            text = re.sub(r'\b[\w\-.]+?@\w+?\.\w{2,4}\b', 'emailaddr', str(text))
            text = re.sub(r'(http[s]?\S+)|(\w+\.[A-Za-z]{2,4}\S*)', 'httpaddr', text)
            text = re.sub(r'£|€|\$', 'currencysymb', text)
            text = re.sub(r'\b(\+\d{1,2}\s)?\d?[\-(.]?\d{3}\)?[\s.-]?\d{3}[\s.-]?\d{4}\b','phonenumbr', text)
            text = re.sub(r'\d+(\.\d+)?', 'numbr', text)
            text = re.sub(r'[^\w\d\s]', ' ', text)
            text = re.sub(r'\s+', ' ', text)
            text = re.sub(r'^\s+|\s+?$', '', text.lower())
            
            processed_text = ' '.join(stemmer.stem(term) for term in text.split() if term not in set(stop_words))
            
            X_to_words.append(processed_text)

        return X_to_words

In [15]:
# email before and after transformation()
print("[email before transformation()]: ")
print(email_to_plain(spam_emails[2]))
print("[email after transformation()]: ")
print(transformation(spam_emails)[2])

[email before transformation()]: 
1) Fight The Risk of Cancer!
http://www.adclick.ws/p.cfm?o=315&s=pk007

2) Slim Down - Guaranteed to lose 10-12 lbs in 30 days
http://www.adclick.ws/p.cfm?o=249&s=pk007

3) Get the Child Support You Deserve - Free Legal Advice
http://www.adclick.ws/p.cfm?o=245&s=pk002

4) Join the Web's Fastest Growing Singles Community
http://www.adclick.ws/p.cfm?o=259&s=pk007

5) Start Your Private Photo Album Online!
http://www.adclick.ws/p.cfm?o=283&s=pk007

Have a Wonderful Day,
Offer Manager
PrizeMama













If you wish to leave this list please use the link below.
http://www.qves.com/trim/?ilug@linux.ie%7C17%7C114258


-- 
Irish Linux Users' Group: ilug@linux.ie
http://www.linux.ie/mailman/listinfo/ilug for (un)subscription information.
List maintainer: listmaster@linux.ie


[email after transformation()]: 
numbr fight risk cancer httpaddr numbr slim guarante lose numbr numbr lb numbr day httpaddr numbr get child support deserv free legal advic httpaddr nu

In [16]:
%%time
print("Transforming email groups...")
transformed_ham_emails = transformation(ham_emails)
transformed_spam_emails = transformation(spam_emails)
transformed_ham_spam_emails = transformation(ham_emails + spam_emails)
print("Transformation done!")

Transforming email groups...
Transformation done!
CPU times: user 35.6 s, sys: 204 ms, total: 35.8 s
Wall time: 36.1 s


##### Tokenization
Individual terms can be tokenized and a bag of words model is generated. Consequently, every sequence of *n* terms called n-grams preserving word order can potentially capture more information than a bag of words.
A n-gram model models sequences, predicts $x_i$ based on previous words $x_{i-1}$, ... , $x_{i-(n-1)}$.

##### Term Frequency Inverse Document Frequency (tf-idf)
After selecting the preferred n-grams, the frequency of each n-gram can be computed by term frequency *tf* and inverse document frequency *idf*.

*tf* computes the frequency of each term, however, some n-grams appear more offen in multiple emails, while some appear less but relatively more in spam emails. To address this, idf calculates the logarithmically inverse fraction of training records that contain the term. 

`TfidfVectorizer()` from the `scikit-learn` library provides the complete framework to select n-grams, compute their tf-idf statistic and transform the data in the compressed sparse row matrix format, which is ready for modeling.

In [17]:
# bigrams are chosen in ngram_range selection
vectorizer = TfidfVectorizer(ngram_range=(2, 2))

In [18]:
%%time
X_ngrams = vectorizer.fit_transform(transformed_ham_spam_emails)

CPU times: user 1.22 s, sys: 49.9 ms, total: 1.27 s
Wall time: 1.24 s


In [19]:
X_ngrams

<3052x194096 sparse matrix of type '<class 'numpy.float64'>'
	with 365990 stored elements in Compressed Sparse Row format>

#### `feature set 2`: "most-frequent-word-count" 
The idea of generating `feature set 2` is different from `feature set 1`, which is based on counting the most frequently occuring words from the email content.

The classes `email_to_words()` and `word_count_to_vector()` follow the existing kernel$^4$. `email_to_words()` serves to transform the plain text to word frequencies through punctuation marks removal, lower case conversion and stemming. `word_count_to_vector()` converts word frequencies to a compressed sparse row matrix format for modeling.

In [20]:
class email_to_words(BaseEstimator, TransformerMixin):
        
    def __init__(self, 
                 lowercaseConversion = True, 
                 punctuationRemoval = True, 
                 stemming = True):
        self.lowercaseConversion = lowercaseConversion
        self.punctuationRemoval = punctuationRemoval
        self.stemming = stemming
        self.stemmer = nltk.PorterStemmer()
        
    def fit(self, X, y = None):
        return self
    
    def transform(self, X, y = None):
        X_to_words = []
        for email in X:
            text = email_to_plain(email)
            
            if text is None:
                text = 'empty'
            
            if self.lowercaseConversion:
                text = text.lower()
                        
            if self.punctuationRemoval:
                text = text.replace('.','')
                text = text.replace(',','')
                text = text.replace('!','')
                text = text.replace('?','')
                
            word_counts = Counter(text.split())
            if self.stemming:
                stemmed_word_count = Counter()
                for word, count in word_counts.items():
                    stemmed_word = self.stemmer.stem(word)
                    stemmed_word_count[stemmed_word] += count
                word_counts = stemmed_word_count
            X_to_words.append(word_counts)
            
        return np.array(X_to_words)

In [21]:
# email before and after email_to_plain()
print("[email before email_to_plain()]: ")
print(email_to_plain(ham_emails[0]))
print("[email after email_to_plain(): ")
print(email_to_words().transform(ham_emails[:1]))

[email before email_to_plain()]: 
    Date:        Wed, 21 Aug 2002 10:54:46 -0500
    From:        Chris Garrigues <cwg-dated-1030377287.06fa6d@DeepEddy.Com>
    Message-ID:  <1029945287.4797.TMDA@deepeddy.vircio.com>


  | I can't reproduce this error.

For me it is very repeatable... (like every time, without fail).

This is the debug log of the pick happening ...

18:19:03 Pick_It {exec pick +inbox -list -lbrace -lbrace -subject ftp -rbrace -rbrace} {4852-4852 -sequence mercury}
18:19:03 exec pick +inbox -list -lbrace -lbrace -subject ftp -rbrace -rbrace 4852-4852 -sequence mercury
18:19:04 Ftoc_PickMsgs {{1 hit}}
18:19:04 Marking 1 hits
18:19:04 tkerror: syntax error in expression "int ...

Note, if I run the pick command by hand ...

delta$ pick +inbox -list -lbrace -lbrace -subject ftp -rbrace -rbrace  4852-4852 -sequence mercury
1 hit

That's where the "1 hit" comes from (obviously).  The version of nmh I'm
using is ...

delta$ pick -version
pick -- nmh-1.0.4 [compiled on fuchs

After preprocessing the emails, the above is a list of words with their frequencies for each email. The next step is to choose which words will be included for the spam detection and which words are excluded.

Since rarely occuring words only exist in a few emails, they may cause the model to overfit (which will be explained in the modeling section) the training data. Hence, only the **most frequently occuring words** will be considered to create a vocabulary list.

In [22]:
class word_count_to_vector(BaseEstimator, TransformerMixin):
    def __init__(self, vocabulary_size = 1000):
        self.vocabulary_size = vocabulary_size
        
    def fit(self, X, y = None):
        total_word_count = Counter()
        for word_count in X: # word_count: whole dictionary in Counter()
            for word, count in word_count.items():
                total_word_count[word] += count
        self.most_common = total_word_count.most_common()[:self.vocabulary_size] # list the most common words and their frequencies
        self.vocabulary_ = {word: index + 1 for index, (word, count) in enumerate(self.most_common)} # list the rank of the most common words
        return self
    
    def transform(self, X, y = None):
        rows = []
        cols = []
        data = []
        for row, word_count in enumerate(X):
            for word, count in word_count.items():
                rows.append(row)
                cols.append(self.vocabulary_.get(word, 0))
                data.append(count)
        return csr_matrix((data, (rows, cols)), shape = (len(X), self.vocabulary_size + 1))

In [23]:
print("[email before word_count_to_vector()]: ")
print(email_to_words().transform(ham_emails[:1]))
print("[email after word_count_to_vector()]: ")
print(word_count_to_vector(vocabulary_size = 1000).fit_transform(email_to_words().transform(ham_emails[:1])).toarray())

[email before word_count_to_vector()]: 
[Counter({'the': 15, 'pick': 9, '-lbrace': 6, 'of': 5, '-rbrace': 5, 'i': 4, 'is': 4, '-list': 4, 'thi': 3, '+inbox': 3, '-subject': 3, 'ftp': 3, '-sequenc': 3, '18:19:04': 3, 'command': 3, 'delta$': 3, 'from': 3, 'error': 2, '18:19:03': 2, '4852-4852': 2, 'mercuri': 2, '1': 2, 'hit': 2, "that'": 2, 'come': 2, 'version': 2, 'use': 2, 'on': 2, 'and': 2, 'one': 2, 'date:': 1, 'wed': 1, '21': 1, 'aug': 1, '2002': 1, '10:54:46': 1, '-0500': 1, 'from:': 1, 'chri': 1, 'garrigu': 1, '<cwg-dated-103037728706fa6d@deepeddycom>': 1, 'message-id:': 1, '<10299452874797tmda@deepeddyvirciocom>': 1, '|': 1, "can't": 1, 'reproduc': 1, 'for': 1, 'me': 1, 'it': 1, 'veri': 1, 'repeat': 1, '(like': 1, 'everi': 1, 'time': 1, 'without': 1, 'fail)': 1, 'debug': 1, 'log': 1, 'happen': 1, 'pick_it': 1, '{exec': 1, '-rbrace}': 1, '{4852-4852': 1, 'mercury}': 1, 'exec': 1, 'ftoc_pickmsg': 1, '{{1': 1, 'hit}}': 1, 'mark': 1, 'tkerror:': 1, 'syntax': 1, 'in': 1, 'express': 1,

In [24]:
email_pipeline = Pipeline([
    ("Email to Words", email_to_words()),
    ("Wordcount to Vector", word_count_to_vector()),
])

### 3. Modeling
A lot of classification algorithms are available. According to the readme documentation of SpamAssassin$^6$, our 2500 non-spam messages belong to easy_ham and they should be easily differentiated from spam. Therefore, instead of using sophisicated and hybrid models, I rely on relatively simple classification algorithms to solve this problem.

In this section a modeling pipeline using different classifiers and feature sets will be run. Also, identification of model underfitting and overfitting will be discussed. To prevent underfitting and overfitting, the modeling results will be evaluated first through the cross-validation score, and then evaluated by evaluation metrics of classification.

#### Dummy ####
The first group of classifier is the Dummy Classifier, which provides measures of "baseline" performances. 
There are three strategy options for the Dummy Classifier. The default strategy is **stratified**. In our case Dummy Classifier will predict that there is a 16% (spam to all ratio) probability that each new record in the test set possesses the target property (spam). 

Another strategy is **most frequent**. In our case since ham emails dominate all emails (84% among all), Dummy Classifier will guess that every new record in the test set gives the majority guess, that is 0. There will be **no false positives (FP)**. This leads to perfect precision.

The third strategy is **constant**. Let us set the constant value to be 1 from the minority group as the counter strategy of most frequent. Since all cases are classified as spams, there will be **no false negatives (FN)**. This leads to perfect recall rate. 

The goal of these three classifiers is to provide the baseline performances based on the given target distribution only. In our modeling task we aim to come up with a model which at least beats the baseline model. Otherwise, we do not have to go into the email at all and extract features for modeling.

#### Logistic Regression (LR) ####
Logistic Regression, aka maximum-entropy classification identifies a set of parameters by maximizing the performance of the classifer, the total likelihood of the training corpus. In other words, it maximizes the likelihood function or entropy to identify parameters which fits the data through the sigmoid transformation. It is a simple method to classify data.


#### Multinomial NB #### 
Naïve Bayes, aka Bayesian Filter, has been known to perform well in the domain of spam detection. It is easy to implement, and its computational complexity and accuracy is comparable to other sophisticated machine learning algorithms. Its simplicity is based on the assumption of conditional independence between each pair of features given the value of the class. 

In n-gram probability model, the probability of a word conditional on some number of previous words follows a multinomial distribution. Counting words is modeling probability of word occurence. Therefore, multinomial distribution is best suitable as the underlying distribution assumption of Naïve Bayes.


#### Support Vector Machine (SVM) - Linear Kernel ####
Support vector machines (SVMs) finds hyperplanes for separation. It has been known to perform well with small datasets. It is not overly influenced by noisy data and not very prone to overfitting. A linear kernel is chosen due to the large amount of features. Using nonlinear kernel would be computational expensive.


#### Random Forest (RF) ####
Random Forest is the ensemble learning technique which aggregates the results of multiple trees. These trees are uncorrelated to each other, therefore random forest is a stronger classifier than a single decision tree. It has been used for phishing email classification before NLP gains its popularity.

#### Underfitting and Overfitting ####
If a model is not sufficient to fit the training data, then the model suffers from underfitting. If a model fits too specific to the training data, it learns the noise, overfits the training data and such a model cannot generalise to unseen data. Overfit can occur due to too many features in a small training dataset or the use of complicated models. 
A perfect (or best) model should be the one which reduces underfitting or overfitting. There are three practices for identification. They are datasets splitting, cross-validation and bootstrap.

One of the most common practices is to **split datasets** into training set, validation set and test set, where training set is used to train on the data and generate the model, validation is used to evaluate the model generated and select the best model, while test set serves as final assertion. This method is subject to the size of the training set. The modeling performance may be affected if the training set is too small.

Another evaluation to quantify underfitting / overfitting is **cross-validation** which generalises the model performance using the same training data. The idea is to split the training dataset into subsets and each subset will be used as hold-out "test" set to validate the trained model. Averaging results will provide final model performance. Estimates from cross-validation is still biased, but bias reduced with the number of folds. 

**Bootstrap** is the advanced version of cross-validation. A simple version is leave-one-out (LOO), in which every single record will be left out in the training set and can be treated as a "test" record to validate the trained model. The model estimate of bootstrap is least biased but it is most computational expensive.

To take a balance among all methodologies, 10-fold cross-validation will be conducted with the reported mean and the standard deviation (std) of cv-score. CV-score in cross-validation is also the accuracy score in the classification task. Mean of the cv-score can be compared with the accuracy of the model, while standard deviation checks how dispersed the estimates are.

#### Evaluation Metrics ####
"Predicted" and "identified" have the same meaning in the following definitions.
- True Positives (TP) are the number of predicted spams which are correctly identified.
- True Negatives (TN) are the number of predicted hams which are correctly identified.
- False Positives (FP) are the number of predicted spams which are incorrectly identified.
- False Negatives (FN) are the number of predicted hams which are incorrectly identified.
- **Accuracy** is the simplest metric to evaluate a classifier which is calculated as $\frac{TP + TN}{TP + TN + FP + FN}$. It measures the percentage of inputs in the test set that the classifier correctly labeled, in our case, the ratio of correctly identified hams and spams out of all emails.
- **Precision** indicates the fraction of relevant items among all the retrieved items$^7$, which is computed as $\frac{TP}{TP + FP}$. This is the correctly identified spams out of all the spams identified.
- **Recall** indicates the fraction of the relevant items that are actually retrieved$^7$, which is computed as $\frac{TP}{TP + FN}$. This is the correctly identified spams out of all actual spams.
- **F1**, aka F-Measure (or F-Score), which combines the precision and recall to give a single score, is defined as the harmonic mean of the precision and recall: $\frac{2 × Precision × Recall}{Precision + Recall}$. In other words, it takes both false positives and false negatives into account.

Setting `random_state = 42` to a specific number is used to initialize a pseudorandom number generator, such that the modeling results are reproducible. 

`stratify = y` in the `train_test_split()` parameter makes a split so that the proportion of values in the sample produced will be the same as the proportion of target values.

In [25]:
def Dummy_Stratified():
    return DummyClassifier(random_state = 42, strategy = "stratified")

def Dummy_Frequent():
    return DummyClassifier(random_state = 42, strategy = "most_frequent")

def Dummy_Constant():
    return DummyClassifier(constant = 1, random_state = 42, strategy = "constant")

def NB_Multinomial():
    return MultinomialNB()
   
def LR():
    return LogisticRegression(solver = "liblinear", random_state = 42)
    
def RF():
    return RandomForestClassifier(random_state = 42)
    
def SVM():
    return svm.LinearSVC(random_state = 42)

switcher = {
    0: Dummy_Stratified,
    1: Dummy_Frequent,
    2: Dummy_Constant,
    3: NB_Multinomial,
    4: LR,
    5: RF,
    6: SVM
}

def model_choice(i):
    func = switcher.get(i, lambda: 'Invalid')
    return func()

In [26]:
def model(model_name, feature):
    result = {}
    
    X = np.array(ham_emails + spam_emails) if feature == "word-count" else X_ngrams
    y = np.array([0] * len(ham_emails) + [1] * len(spam_emails))

    X_train, X_test, y_train, y_test = train_test_split(X, 
                                                        y, 
                                                        test_size = 0.2, 
                                                        random_state = 42, 
                                                        stratify = y)
    
    # get modeling instance through model choice by model_name
    clf = model_choice({v:k for k, v in switcher.items()}[model_name])
    
    # fit_transform() joins word-count-fit and transform x as initial step
    X_augmented_train = email_pipeline.fit_transform(X_train) if feature == "word-count" else X_train

    # run CV to guarantee consistent performance
    # dense array is required for NB
    cv_score = cross_val_score(clf, X_augmented_train, y_train, cv = 10) if model_name.__name__[0:2] != "NB" else cross_val_score(clf, X_augmented_train.toarray(), y_train, cv = 10)     
    cv_score_mean = cv_score.mean()
    cv_score_sd = cv_score.std()
    
    # transform() apply tranformation on the fitted internal object   
    X_augmented_test = email_pipeline.transform(X_test) if feature == "word-count" else X_test
    clf.fit(X_augmented_train, y_train) if model_name.__name__[0:2] != "NB" else clf.fit(X_augmented_train.toarray(), y_train)
    
    y_pred = clf.predict(X_augmented_test) if model_name.__name__[0:2] != "NB" else clf.predict(X_augmented_test.toarray())

    ac = accuracy_score(y_test, y_pred)    
    pc = precision_score(y_test, y_pred)
    rc = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    result = {'feature': feature,
              'model_name': model_name, 
              'clf': clf,
              'cv_score': cv_score,
              'accuracy': ac,
              'precision': pc,
              'recall': rc,
              'F1': f1,
              'y_test': y_test,
              'y_pred': y_pred
             }
    
    return result

In [27]:
def model_result(model_seq, feature):
    data = []
    for j in model_seq:
        m = model(j, feature)
        data.append([m['feature'],
                    m['model_name'].__name__,
                    float("{0:.4f}".format(m['cv_score'].mean())), 
                    float("{0:.4f}".format(m['cv_score'].std())), 
                    float("{0:.4f}".format(m['accuracy'])), 
                    float("{0:.4f}".format(m['precision'])), 
                    float("{0:.4f}".format(m['recall'])), 
                    float("{0:.4f}".format(m['F1']))])
        df = pd.DataFrame(data)
        df.columns = ['feature', 'model_name', 'cv_score_mean', 'cv_score_std', 'accuracy', 'precision', 'recall', 'F1']
    return df

### 4. Results
In this section I am going to discuss the results based on the above modeling pipeline.

In [28]:
warnings.filterwarnings("ignore")

In [29]:
%%time
model_result_dummy = model_result({Dummy_Stratified: 'Dummy_Stratified', Dummy_Frequent: 'Dummy_Frequent', Dummy_Constant: 'Dummy_Constant'}, "-")

CPU times: user 113 ms, sys: 9.2 ms, total: 122 ms
Wall time: 122 ms


In [30]:
%%time
model_result_wc = model_result({NB_Multinomial: 'NB_Multinomial', LR: 'LR', RF: 'RF', SVM: 'SVM'}, "word-count")

CPU times: user 56 s, sys: 469 ms, total: 56.4 s
Wall time: 56.3 s


In [31]:
%%time
model_result_stopword_ngram_tdidf = model_result({NB_Multinomial: 'NB_Multinomial', LR: 'LR', RF: 'RF', SVM: 'SVM'}, "stopword + n-gram + td-idf")

CPU times: user 1min 8s, sys: 1min 2s, total: 2min 10s
Wall time: 2min 10s


In [32]:
model_result_dummy

,feature,model_name,cv_score_mean,cv_score_std,accuracy,precision,recall,F1
0,-,Dummy_Stratified,0.7432,0.0177,0.7381,0.2222,0.24,0.2308
1,-,Dummy_Frequent,0.8357,0.0010,0.8363,0.0000,0.00,0.0000
2,-,Dummy_Constant,0.1643,0.0010,0.1637,0.1637,1.00,0.2813


In [33]:
model_result_wc

,feature,model_name,cv_score_mean,cv_score_std,accuracy,precision,recall,F1
0,word-count,NB_Multinomial,0.9803,0.0073,0.9787,0.9579,0.91,0.9333
1,word-count,LR,0.9865,0.0080,0.9885,0.9895,0.94,0.9641
2,word-count,RF,0.9767,0.0082,0.9755,0.9570,0.89,0.9223
3,word-count,SVM,0.9791,0.0097,0.9853,0.9505,0.96,0.9552


In [34]:
model_result_stopword_ngram_tdidf

,feature,model_name,cv_score_mean,cv_score_std,accuracy,precision,recall,F1
0,stopword + n-gram + td-idf,NB_Multinomial,0.9222,0.0162,0.9394,1.0000,0.63,0.7730
1,stopword + n-gram + td-idf,LR,0.8849,0.0094,0.8903,1.0000,0.33,0.4962
2,stopword + n-gram + td-idf,RF,0.9062,0.0171,0.8887,0.6159,0.85,0.7143
3,stopword + n-gram + td-idf,SVM,0.9509,0.0153,0.9591,1.0000,0.75,0.8571


##### Model: Dummy
The Dummy models serve the base models which rely on the distribution of the target variable (spam or ham) only. They are independent of our selected features. 

The Stratified Dummy predicts 84% of emails ham and 16% of emails spam gives the accuracy of 74%. 

The Most Frequent Dummy predicts all emails ham which predicts all hams perfect but has no predictive power at all in spam predictions. True Positives are all zeros, which make precision, recall and F1 all undefined, even 0 is the default output for undefined values in the `sklearn.metrics`. Its high accuracy is due to the high ratio of hams out of all emails.

The Constant Dummy 1 predicts every email as spam and makes it a perfect spam detector but has no predictive power at all in predicting hams. 

In this modeling task, we hope to build a classifier with model performance at least better than the base model performance, which is 0.8357 in accuracy.

##### `Feature Set 1`: stopwords + n-gram + td-idf
Among all models using the `feature 1` **stopwords + n-gram + td-idf**, **Support Vector Machine** with **linear kernel** gives the best model performance in terms of **accuracy and F1**. 

##### `Feature Set 2`: most-frequent-word-count
Among all models using the `feature 2` **most-frequent-word-count**, **Logistic Regression** followed by multinomial Naïve Bayes gives the best model performance in terms of **accuracy, precision, and F1**, though multinomial Naïve Bayes gives a smaller standard deviation of cross-validation score than logistic regression (0.0073 vs 0.008), suggesting that logistic regression may suffer a bit from overfitting. Mean of cross-validation scores are quite close to the accuracy, suggesting that all models are not suffering from underfitting.

##### Feature Comparison: stopwords + n-gram + td-idf vs most-frequent-word-count
All the models based on `feature set 2` **most-frequent-word-count** have a higher accuracy and F1 score than those based on the `feature set 1` **stopwords + n-gram + td-idf**. In the `feature set 1` **stopwords + n-gram + td-idf**, except random forest, precision from the other models has the perfect precision rate.

### 5. Conclusions 
Which is the best classifier? It depends on the application of spam email detector. 

If the use case is to introduce a beta version of an email spam detector like **no-spam in inbox**, then false negatives should be minimized and precision will be the most important evaluation metric. In this case, the model: **Support Vector Machine** with **linear kernel** from the `feature set 1` **stopwords + n-gram + tfidf** serves this suppose.

If the use case is to introduce an email spam detector to **reduce bad user experience** in searching for important emails from junk mailbox and filtering spams from indox, then accuracy and F1 give a balance of reducing false postivies and false negatives. In this case **Logistic Regression** with `feature set 2` **most-frequent-word-count** gives a better user experience in general.

### 6. Improvements
This is the very initial version of the Spam Email Detection. Based on the above discussions, the following points can be taken further into account.

- Features
    - Some spam emails are dominated by capital letters. In our text preprocessing all letters are converted into  small letters which uncover potential signals in spam prediction. The ratio of capital letters in text can be used as feature.
    - Currently features are extracted based on simple rules and regular expressions. More sophisticated features can be explored$^8$.
    - Compare with `feature set 2` **most-frequent-word-count**, the current `feature set 1` **stopwords + n-gram + tf-idf** is not performing so well. One suggestion is to refine words in the stopwords list. Another suggestion is to try other stemmers, because the current PorkerStemmer is a relatively gentle stemming algorithm and its development is paused.
    
- Modeling
    - Grid search CV can be used after model selection to optimise hyperparameters in model tuning.
    
- Evaluation
    - To compare models with similar performances, confusion matrix can be used to explore instances of false positives and false negatives. 
    
- Code Refactoring
    - Generation of `feature set 1` and `feature set 2` is written based on different data structures. If the developed text preprocessing pipeline is useful in reviewing features, the code refactoring is needed in text preprocessing.

### 7. Appendix

$^1$ https://www.wordstream.com/blog/ws/2017/06/29/email-marketing-statistics

$^2$ https://www.kaggle.com/veleon/ham-and-spam-dataset

$^3$ https://spamassassin.apache.org/

$^4$ https://www.kaggle.com/veleon/spam-classification

$^5$ https://en.wikipedia.org/wiki/Email_spam

$^6$ https://spamassassin.apache.org/old/publiccorpus/readme.html

$^7$ https://en.wikipedia.org/wiki/Precision_and_recall

$^8$ https://stackoverflow.com/questions/770238/neural-networks-for-email-spam-detection